#### Este baseline se basara en armar un modelo LSTM por cada producto, con una optimizacion de hiper parametros escueta, para poder comparar con futuros experimientos. En caso de que esta alternativa funcione bien, seria recomendable incorporar parametros de optimizacion extra.

#### Imports

In [9]:
import warnings
warnings.filterwarnings('ignore')
from prophet import Prophet

import pandas as pd
import numpy as np

In [10]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')
final_dataset['periodo'] = final_dataset['periodo'].astype(str)
final_dataset['year'] = final_dataset['periodo'].str[:4].astype(int)
final_dataset['month'] = final_dataset['periodo'].str[5:7].astype(int)

# Seleccionar noviembre y diciembre de 2017
nov_diciembre_2017 = (final_dataset['year'] == 2017) & ((final_dataset['month'] == 11) | (final_dataset['month'] == 12))

# Seleccionar todo el año 2018 excepto noviembre y diciembre
resto_2018 = (final_dataset['year'] == 2018) & ((final_dataset['month'] != 11) & (final_dataset['month'] != 12))

# Combinar las condiciones con el operador lógico OR (|)
filtro = nov_diciembre_2017 | resto_2018

# Aplicar el filtro al dataset
final_dataset = final_dataset[filtro]

# Eliminar la columna 'year'
final_dataset = final_dataset.drop(columns=['year'])

In [11]:
final_dataset.head()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age,month
12,201801,20001,0,437,1256.01136,1169.07532,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,12,1
13,201802,20001,0,302,1150.37849,1043.76470,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,13,2
14,201803,20001,0,591,1902.79056,1856.83534,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,14,3
15,201804,20001,0,384,1286.12277,1251.28462,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,15,4
16,201805,20001,0,456,1303.62129,1293.89788,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,16,5


In [12]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter','y']

In [13]:
duplicates = final_dataset.duplicated(subset=['product_id', 'periodo'])
duplicate_rows = final_dataset[duplicates]

if duplicate_rows.empty:
    print("No hay registros duplicados por product_id y periodo.")
else:
    print("Registros duplicados encontrados por product_id y periodo:")
    display(duplicate_rows)


No hay registros duplicados por product_id y periodo.


#### Funcion para crear los modelos

#### Armado de los modelos

In [14]:
final_dataset['periodo'] = final_dataset['periodo'].astype(str)
final_dataset['periodo'] = pd.to_datetime(final_dataset['periodo'], format='%Y%m', errors='coerce')

In [15]:
from prophet import Prophet
import numpy as np
import pandas as pd
import joblib
import os

product_ids = final_dataset['product_id'].unique()
predictions = []

for product_id in product_ids:
    product_data = final_dataset[final_dataset['product_id'] == product_id].sort_values(by='periodo')[['periodo', 'y']]
    
    product_data = product_data.rename(columns={'periodo': 'ds', 'y': 'y'})
    #product_data['y'] = np.log1p(product_data['y'])
    
    
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        n_changepoints=20,
        changepoint_prior_scale=0.05
    )
    model.fit(product_data)
    
    future = model.make_future_dataframe(periods=2)
    forecast = model.predict(future)
    
    #forecast['yhat'] = np.expm1(forecast['yhat'])
    forecast['yhat'] = forecast['yhat'].clip(lower=0)
    
    os.makedirs('Prophet_results', exist_ok=True)
    joblib.dump(model, f'Prophet_results/model_product_{product_id}.pkl')
    print(forecast.head())  # Imprime las primeras filas del DataFrame forecast
    
    predicted_y = forecast.iloc[-1]['yhat']
    predicted_y = np.where(predicted_y < 0, 0, predicted_y)
    predictions.append({'product_id': product_id, 'predicted_y': predicted_y})
    
    print(f'Modelo para el producto {product_id} entrenado y guardado. Predicción a 2 meses: {predicted_y}. Número de predicciones: {len(predictions)}')


21:35:26 - cmdstanpy - INFO - Chain [1] start processing
21:35:27 - cmdstanpy - INFO - Chain [1] done processing


          ds        trend   yhat_lower   yhat_upper  trend_lower  trend_upper   
0 2018-01-01  2499.723442  1169.061559  1169.061560  2499.723442  2499.723442  \
1 2018-02-01  2308.259751  1043.770937  1043.770937  2308.259751  2308.259751   
2 2018-03-01  2135.324804  1856.848799  1856.848799  2135.324804  2135.324804   
3 2018-04-01  1943.861112  1251.276058  1251.276059  1943.861112  1943.861112   
4 2018-05-01  1758.573667  1293.869968  1293.869968  1758.573667  1758.573667   

   additive_terms  additive_terms_lower  additive_terms_upper       yearly   
0    -1330.661883          -1330.661883          -1330.661883 -1330.661883  \
1    -1264.488814          -1264.488814          -1264.488814 -1264.488814   
2     -278.476005           -278.476005           -278.476005  -278.476005   
3     -692.585053           -692.585053           -692.585053  -692.585053   
4     -464.703700           -464.703700           -464.703700  -464.703700   

   yearly_lower  yearly_upper  multiplicativ

21:35:27 - cmdstanpy - INFO - Chain [1] start processing
21:35:29 - cmdstanpy - INFO - Chain [1] done processing


          ds       trend   yhat_lower   yhat_upper  trend_lower  trend_upper   
0 2018-01-01 -112.398665   984.797657   984.797659  -112.398665  -112.398665  \
1 2018-02-01   41.090542   712.001973   712.001975    41.090542    41.090542   
2 2018-03-01  179.725955   966.865447   966.865449   179.725955   179.725955   
3 2018-04-01  333.215162   999.215576   999.215578   333.215162   333.215162   
4 2018-05-01  481.753104  1103.384042  1103.384044   481.753104   481.753104   

   additive_terms  additive_terms_lower  additive_terms_upper       yearly   
0     1097.196323           1097.196323           1097.196323  1097.196323  \
1      670.911432            670.911432            670.911432   670.911432   
2      787.139493            787.139493            787.139493   787.139493   
3      666.000415            666.000415            666.000415   666.000415   
4      621.630939            621.630939            621.630939   621.630939   

   yearly_lower  yearly_upper  multiplicative_term

21:35:29 - cmdstanpy - INFO - Chain [1] start processing
21:35:36 - cmdstanpy - INFO - Chain [1] done processing
21:35:36 - cmdstanpy - INFO - Chain [1] start processing


...

22:20:28 - cmdstanpy - INFO - Chain [1] done processing
22:20:28 - cmdstanpy - INFO - Chain [1] start processing


          ds     trend  yhat_lower  yhat_upper  trend_lower  trend_upper   
0 2018-01-01  4.062777    1.736714    1.736714     4.062777     4.062777  \
1 2018-02-01  5.188983    2.456802    2.456802     5.188983     5.188983   
2 2018-03-01  6.206200   11.203202   11.203202     6.206200     6.206200   
3 2018-04-01  7.332406   10.197083   10.197083     7.332406     7.332406   
4 2018-05-01  8.422282   14.155103   14.155104     8.422282     8.422282   

   additive_terms  additive_terms_lower  additive_terms_upper    yearly   
0       -2.326063             -2.326063             -2.326063 -2.326063  \
1       -2.732181             -2.732181             -2.732181 -2.732181   
2        4.997002              4.997002              4.997002  4.997002   
3        2.864677              2.864677              2.864677  2.864677   
4        5.732822              5.732822              5.732822  5.732822   

   yearly_lower  yearly_upper  multiplicative_terms   
0     -2.326063     -2.326063        

22:20:49 - cmdstanpy - INFO - Chain [1] done processing
22:20:50 - cmdstanpy - INFO - Chain [1] start processing


          ds     trend  yhat_lower  yhat_upper  trend_lower  trend_upper   
0 2018-01-01  0.838152    4.515484    4.515484     0.838152     0.838152  \
1 2018-02-01  0.813717    4.484607    4.484607     0.813717     0.813717   
2 2018-03-01  0.794028    7.301852    7.301852     0.794028     0.794028   
3 2018-04-01  0.731501    4.608406    4.608406     0.731501     0.731501   
4 2018-05-01  1.146392    4.851791    4.851791     1.146392     1.146392   

   additive_terms  additive_terms_lower  additive_terms_upper    yearly   
0        3.677333              3.677333              3.677333  3.677333  \
1        3.670890              3.670890              3.670890  3.670890   
2        6.507824              6.507824              6.507824  6.507824   
3        3.876904              3.876904              3.876904  3.876904   
4        3.705398              3.705398              3.705398  3.705398   

   yearly_lower  yearly_upper  multiplicative_terms   
0      3.677333      3.677333        

KeyboardInterrupt: 

In [8]:
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('../../Datasets/predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv.')

Todas las predicciones han sido generadas y guardadas en predictions.csv.
